In [1]:
import csv
import json
import re
import collections
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pprint
pp=pprint.PrettyPrinter(indent=4)

In [2]:
# csvfile = open('ubic_click_action_1013.txt', 'r') # 2랑 차이는 뭘까 ? 2는 주피터에서 실행이 안된다!

# reader = pd.read_csv('./ubic_click_action_1013.txt')
# reader

file = open('all_ubic_click_action_1013.txt',mode='r').read()
# file

In [3]:
import json
file_json = json.loads(file)
# dict(file)

In [4]:
type(file_json)

list

In [5]:
# file_json

In [6]:
columns = list(file_json[0].keys()) + ['actionType_score'] # column name list
print(columns)
# now, actionType 을 인코딩해야한다

#     emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

#     emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
#     emotion_df['emotion_encoding'] = pd.factorize(emotion_df['emotion_keyword'])[0]  # pd.factorize(df['Col'])[0]

#     #print(emotion_df.values)
#     emotion_keyword_to_vec = {} #딕셔너리로 저장하면 편하겠지?
#     for keyword,encoding in emotion_df.values:
#         #print(keyword,encoding)
#         emotion_keyword_to_vec[keyword] = encoding
score = {
    'cart-create': 3 ,
    'order-create': 5,
    'click': 2 ,
    'hover': 1,
    'order-cancel': -6,
    'cart-cancel': -4,

    'cart-modify-plus': 3,
    'cart-modify-minus': -3,
}        

columns_value_list = []
# set_action_type = set()

for row in file_json:
#     print(row.keys())
    
    # 왼쪽부터 순차적으로 검사하나보다
    if str(row['userId']).isdigit() and str(row['productId']).isdigit() and 'categoryId' in row and str(row['categoryId']).isdigit():
#         row['now']
        row['userId'] = int(row['userId'])
        row['productId'] = int(row['productId'])
        row['categoryId'] = int(row['categoryId'])
        if row['actionType'] in score:
            row['actionType_score'] = score[row['actionType']]
        else:
            row['actionType_score'] = 0
        
#         row['actionType']
#         set_action_type.add(row['actionType'])
#         print(row.values())
        columns_value_list.append(list(row.values()))

['now', 'userId', 'productId', 'categoryId', 'actionType', 'actionType_score']


In [7]:
# print(columns)
# columns_value_list


In [8]:
base_df = pd.DataFrame(columns_value_list, columns=columns)
base_df

,now,userId,productId,categoryId,actionType,actionType_score
0,2020-09-09T23:56:42.276936800,48418,9831,10,hover,1
1,2020-09-09T23:56:42.829949800,48418,11836,12,hover,1
2,2020-09-09T23:56:43.518937100,48418,11836,12,hover,1
3,2020-09-09T23:56:44.659936400,48418,9831,10,hover,1
4,2020-09-09T23:56:45.349936300,48418,11836,12,hover,1
...,...,...,...,...,...,...
2457,2020-10-13T16:42:02.919329300,48089,5424,6,hover,1
2458,2020-10-13T16:42:03.067323100,48089,45417,42,hover,1
2459,2020-10-13T16:42:03.214324,48089,20396,18,hover,1
2460,2020-10-13T16:42:03.546323300,48089,18616,17,hover,1


In [9]:
base_df['now_encoding'] = pd.factorize(base_df['now'])[0]

now_df = base_df[['now', 'now_encoding']]
now_encoding_to_keyword = {} #딕셔너리로 인코딩 정보 저장
for keyword,encoding in now_df.values:
    now_encoding_to_keyword[encoding] = keyword

In [10]:
# userId, productId, actionType
w_actionType_df = base_df[['userId', 'productId', 'actionType_score']]
# w_categoryId_df

actionType_matrix_itemCF = w_actionType_df.pivot_table('actionType_score', index='userId', columns='productId')
actionType_matrix_itemCF = actionType_matrix_itemCF.fillna(0)
# type(actionType_matrix) # DataFrame 


# request product id list 루프 돌며
'''
actionType_matrix[109]
actionType_matrix.corrwith(actionType_matrix[109])
'''
actionType_matrix_itemCF

productId,106,107,108,109,116,124,125,130,131,145,...,38150,38165,45353,45417,45552,45962,46006,46176,46746,46761
userId,,,,,,,,,,,,,,,,,,,,,
614,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1011,0.0,0.0,0.0,1.0,1.0,0.857143,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1013,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1014,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1015,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48441,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48447,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48453,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# userId - userId[0] - productId : userCF
w_actionType_df = base_df[['userId', 'productId', 'actionType_score']]
# w_categoryId_df

actionType_matrix_userCF = w_actionType_df.pivot_table('actionType_score', index='productId', columns='userId')
actionType_matrix_userCF = actionType_matrix_userCF.fillna(0)
# type(actionType_matrix) # DataFrame 
actionType_matrix_userCF

userId,614,1011,1013,1014,1015,1016,1017,1018,1024,1025,...,48427,48431,48433,48439,48440,48441,48447,48453,48454,50658
productId,,,,,,,,,,,,,,,,,,,,,
106,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46176,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
dict()#.keys()
# userId - userId list
sim_userId = actionType_matrix_userCF.corrwith(actionType_matrix_userCF[1011]).sort_values(ascending=False).index[1]
print(sim_userId)

# actionType_matrix_userCF[1011].sort_values(ascending=False)[:2] #.index[:2]
actionType_matrix_userCF[sim_userId].sort_values(ascending=False).index[0]

1025


130

In [13]:
# productId - productId : itemCF

# now 인코딩
# result_factorize = pd.factorize(base_df['now'])
# # result_factorize[0][:500]
# list(result_factorize[1]) # before_encoding


    
# base_df

dict(actionType_matrix_itemCF.corrwith(actionType_matrix_itemCF[109]).sort_values(ascending=False)[:4]).keys() # 내림차 정렬해서 4개만!
# 그러면 userID list 잖아 ?

item_similarity = cosine_similarity(actionType_matrix_userCF)
item_sim_df = pd.DataFrame(data=item_similarity, index=actionType_matrix_userCF.index, columns=actionType_matrix_userCF.index) # 행으로 연산하는거 맞다
item_sim_df

productId,106,107,108,109,116,124,125,130,131,145,...,38150,38165,45353,45417,45552,45962,46006,46176,46746,46761
productId,,,,,,,,,,,,,,,,,,,,,
106,1.000000,0.707107,0.577350,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
107,0.707107,1.000000,0.816497,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
108,0.577350,0.816497,1.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
109,0.000000,0.000000,0.000000,1.0,1.0,0.608424,1.0,0.57735,0.5,0.57735,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
116,0.000000,0.000000,0.000000,1.0,1.0,0.608424,1.0,0.57735,0.5,0.57735,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45962,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0
46006,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0
46176,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.188982,0.317958,0.629941,0.0,0.0,1.0,0.0,0.0


In [14]:
# item id - item id list
list(item_sim_df[106].sort_values(ascending=False)[1:1+4].index)

[107, 108, 2123, 3723]

In [15]:
actionType_sim = cosine_similarity(actionType_matrix, actionType_matrix)

NameError: name 'actionType_matrix' is not defined

In [ ]:
# actionType 인코딩
result_factorize = pd.factorize(base_df['actionType'])
result_factorize[0][:500]
list(result_factorize[1]) # before_encoding

base_df['actionType_encoding'] = pd.factorize(base_df['actionType'])[0]

actionType_df = base_df[['actionType', 'actionType_encoding']]
actionType_encoding_to_keyword = {} #딕셔너리로 인코딩 정보 저장
for keyword,encoding in actionType_df.values:
#     print(keyword,encoding)
    actionType_encoding_to_keyword[encoding] = keyword
    
base_df

In [ ]:
list(set(base_df['actionType'].values))

In [ ]:
# userId, productId, categoryId
w_categoryId_df = base_df[['userId', 'productId', 'categoryId']]
# w_categoryId_df

categoryId_matrix = w_categoryId_df.pivot_table('categoryId', index='userId', columns='productId')
categoryId_matrix = categoryId_matrix.fillna(0)
categoryId_matrix

In [ ]:
# userId, productId, time
w_now_df = base_df[['userId', 'productId', 'now_encoding']]
# w_categoryId_df

now_matrix = w_now_df.pivot_table('now_encoding', index='userId', columns='productId')
now_matrix = now_matrix.fillna(0)
now_matrix

In [ ]:
# userId, productId, actionType
w_actionType_df = base_df[['userId', 'productId', 'actionType_encoding']]
# w_categoryId_df

actionType_matrix = w_actionType_df.pivot_table('actionType_encoding', index='userId', columns='productId')
actionType_matrix = actionType_matrix.fillna(0)
actionType_matrix

In [ ]:
## user consine sim


categoryId_sim = cosine_similarity(categoryId_matrix, categoryId_matrix)
now_sim = cosine_similarity(now_matrix, now_matrix)
actionType_sim = cosine_similarity(actionType_matrix, actionType_matrix)

In [ ]:
# 행렬 sum
sim_sum = categoryId_sim + now_sim + actionType_sim # ndarray
weather_sim_df = pd.DataFrame(data=weather_sim, index=speed_matrix.index , columns=speed_matrix.index ) / 3
weather_sim_df

인코딩
- one-hot encodings'로 변환시키는 pd.get_dummies 함수
- 카테고리 숫자형 인코딩: pd.factorize 

In [ ]:
# 테스트 코드
# action_type_encoding, action_type_restore = pd.factorize(df_order_create_data['actionType']) 
# action_type_encoding

df_order_create_data['userId'], userId_restore_Index = pd.factorize(df_order_create_data['userId']) 

In [ ]:
df_order_create_data
# productId 를 뽑을 수도 있고
# cateogryId 를 뽑을 수도 있다

In [ ]:
df_order_create_data['actionType'], actionType_restore_Index = pd.factorize(df_order_create_data['actionType']) 
df_order_create_data

In [ ]:
actionType_df = df_order_create_data[['userId','productId','actionType']]
actionType_df
# kurly id 를 넣었어야 했는데, local db id 를 넣어서
# 원격 db 올렸을 때 다 지워야 한다

In [ ]:
actionType_matrix = actionType_df.pivot_table('actionType', index='productId', columns='userId') 

## 다양하게 해서 변경 여지가 있다 ! 그리고 이전에 살펴본 책 다시 읽기 !
# 지금은 mean 이라는 이상한 연산을 적용했는데 count 그리고 actionType 별 다르게 적용해야 한다!
# 보고서 다시 살펴보면서 
actionType_matrix = actionType_matrix.fillna(0)
actionType_matrix

In [ ]:
test_pivot = pd.pivot_table(actionType_df, index = ['userId','productId'], values = 'actionType', aggfunc = 'count')
test_pivot

In [ ]:
# print(type(test_pivot))
# test_pivot.index # MultiIndex

In [ ]:
test_pivot_sim = cosine_similarity(test_pivot, test_pivot)
test_pivot_sim_df = pd.DataFrame(data=test_pivot_sim, index=test_pivot.index , columns=test_pivot.index )
test_pivot_sim_df

In [ ]:
actionType_sim = cosine_similarity(actionType_matrix, actionType_matrix)
actionType_sim_df = pd.DataFrame(data=actionType_sim, index=actionType_matrix.index , columns=actionType_matrix.index )
actionType_sim_df
# 치명적이게도 input: productId > output: productId 으로 작동

In [ ]:
productId_list = actionType_sim_df.columns.tolist() # 열 이름 list
return_dict = {}
for name in productId_list: # 열 이름 루프돌면서
    input_index = name
    temp_df = actionType_sim_df[ (actionType_sim_df[input_index].index!=input_index)] # 행.index != 열.index 인 열을 추출
    
    return_dict[name] = temp_df[input_index].sort_values(ascending=False).index.tolist()[0] # 값 중 최댓값 하나 : 내림차 ascending=False
#     print(type(return_dict[name]))
#     print(name, return_dict[name])

In [ ]:
# 결과
return_dict

In [ ]:
## 디테일 데이터 확인하기
actionType_sim